<a href="https://colab.research.google.com/github/EA-park/AIFFEL/blob/main/%5BExp_06%5DLyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#루브릭

1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?
 - 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?

2. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
 - 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?

3. 텍스트 생성모델이 안정적으로 학습되었는가?
 - 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?